In [13]:
import pandas as pd
import numpy as np
import sys
import os

#change to your own path for python wrapper of xgboost library
sys.path.append(os.path.realpath('./3rdParty/XGBoost/wrapper/'))
import xgboost as xgb

In [14]:
def matchcoor(a, b):
    i1=pd.match(a.loc[:,'chr'],b.loc[:,'chr'])!=-1
    i2=(pd.match(a.loc[:,'chr'],b.loc[:1,'chr'])!=-1) * (a.loc[:,'pos']<b.loc[0,'pos'])
    i3=(pd.match(a.loc[:,'chr'],b.loc[b.shape[0]-1:,'chr'])!=-1) * (a.loc[:,'pos']>b.loc[b.shape[0]-1,'pos'])
    i=i1&(~i2)&(~i3)
    return np.where(~i)[0],np.where(i)[0]

In [15]:
#Load data 
#for this example, we use GWAS Catalog positive variants and matched negative variant with 31kbp mean distance to positives
datalabel=pd.read_csv('example_gwascat_31kbpneg.csv.gz')

#data include 1842 DeepSEA chromatin prediction and evolutionary features 
#(919 relative difference features, 919 abosulte difference features, 4 evolutionary information features)
data=np.asarray(datalabel.iloc[:,6:])
data=np.abs(data)
label=np.asarray(datalabel['name'])

#initialize dataframe for saving cross validation predictions
dataframe=datalabel.iloc[:,:6]
dataframe['score']=0
dataframe['fold']=0

In [16]:
from sklearn.metrics import *
from sklearn.cross_validation import *
from sklearn.preprocessing import *



scores=[]
lms=[]

x,y,d =data,label,dataframe

# Standarsize features
sder=StandardScaler().fit(np.abs(x[:,:]))
# create cross validation folds
# we set shuffle to be false so folds are contiguous regions
kf=KFold(d.shape[0],n_folds=10,shuffle=False,random_state=0)


coor=d.sort(columns=['chr','pos'])
coor=coor.reset_index(drop=True)

fold=-1
for train_i,test_i in kf:
    fold=fold+1

    coortest=coor.iloc[test_i,:]
    coortest=coortest.reset_index(drop=True)
    train_i,test_i=matchcoor(d,coortest)

    #train
    a_train=x[train_i,:]
    b_train=y[train_i]
    a_test=x[test_i,:]
    b_test=y[test_i]
    dtest = xgb.DMatrix( sder.transform(np.abs(a_test)[:,:])[:,:],label=b_test)

    #use weights to balance postive and negative samples
    weights=np.ones(b_train.shape)
    weights[b_train==0]= 1 * 1.0/(np.sum(b_train==0)*1.0/np.sum(b_train==1))
    weights=weights/np.max(weights)
    
    #convert to xgboost format
    dtrain = xgb.DMatrix( sder.transform(np.abs(a_train)), label=b_train,weight=weights)

    param = {'booster':'gblinear','lambda':100,'alpha':0, 'bst:eta':0.1, 'silent':0, 'objective':'reg:logistic', 'nthread':70, 'eval_metric':'auc'}
    lm = xgb.train( param, dtrain, 50, [(dtest,'eval'), (dtrain,'train')])


    #predict

    _,idata=matchcoor(dataframe,coortest)
    dtest = xgb.DMatrix( sder.transform(np.abs(data[idata,:])[:,:]),label=label[idata])
    pred_prob = lm.predict( dtest)
    score = roc_auc_score(y_score=pred_prob,y_true=label[idata])
    scores.append(score)
    dataframe.loc[idata,'score'] = pred_prob
    dataframe.loc[idata,'fold'] = fold


    lms.append(lm)
    print scores


[0]	eval-auc:0.640118	train-auc:0.648752
[1]	eval-auc:0.648300	train-auc:0.659172
[2]	eval-auc:0.654064	train-auc:0.665548
[3]	eval-auc:0.657320	train-auc:0.669716
[4]	eval-auc:0.658730	train-auc:0.672961
[5]	eval-auc:0.659904	train-auc:0.675550
[6]	eval-auc:0.660508	train-auc:0.677594
[7]	eval-auc:0.661056	train-auc:0.679341
[8]	eval-auc:0.661525	train-auc:0.680886
[9]	eval-auc:0.661894	train-auc:0.682272
[10]	eval-auc:0.662348	train-auc:0.683517
[11]	eval-auc:0.662430	train-auc:0.684703
[12]	eval-auc:0.662669	train-auc:0.685726
[13]	eval-auc:0.662878	train-auc:0.686682
[14]	eval-auc:0.662839	train-auc:0.687601
[15]	eval-auc:0.662799	train-auc:0.688463
[16]	eval-auc:0.662768	train-auc:0.689285
[17]	eval-auc:0.662677	train-auc:0.690081
[18]	eval-auc:0.662784	train-auc:0.690782
[19]	eval-auc:0.662537	train-auc:0.691485
[20]	eval-auc:0.662427	train-auc:0.692170
[21]	eval-auc:0.662303	train-auc:0.692802
[22]	eval-auc:0.662318	train-auc:0.693389
[23]	eval-auc:0.662266	train-auc:0.694023
[2

[0.6595429430941081]
[0.6595429430941081, 0.65547968390468792]

[0]	eval-auc:0.619902	train-auc:0.650572
[1]	eval-auc:0.628908	train-auc:0.661077
[2]	eval-auc:0.633834	train-auc:0.667603
[3]	eval-auc:0.636411	train-auc:0.671800
[4]	eval-auc:0.638150	train-auc:0.675156
[5]	eval-auc:0.638996	train-auc:0.677742
[6]	eval-auc:0.639642	train-auc:0.679746
[7]	eval-auc:0.640098	train-auc:0.681527
[8]	eval-auc:0.640329	train-auc:0.682997
[9]	eval-auc:0.640464	train-auc:0.684334
[10]	eval-auc:0.640551	train-auc:0.685546
[11]	eval-auc:0.640720	train-auc:0.686625
[12]	eval-auc:0.640982	train-auc:0.687625
[13]	eval-auc:0.641005	train-auc:0.688550
[14]	eval-auc:0.641024	train-auc:0.689388
[15]	eval-auc:0.641108	train-auc:0.690213
[16]	eval-auc:0.641174	train-auc:0.690952
[17]	eval-auc:0.641154	train-auc:0.691729
[18]	eval-auc:0.641200	train-auc:0.692411
[19]	eval-auc:0.641098	train-auc:0.693077
[20]	eval-auc:0.641182	train-auc:0.693731
[21]	eval-auc:0.641301	train-auc:0.694315
[22]	eval-auc:0.641350	train-auc:0.694892
[23]	eval-auc:0.641393	train-auc:0.695461
[2


[0.6595429430941081, 0.65547968390468792, 0.64236845740901549]

[0]	eval-auc:0.658554	train-auc:0.646505
[1]	eval-auc:0.666713	train-auc:0.657209
[2]	eval-auc:0.669478	train-auc:0.663630
[3]	eval-auc:0.670156	train-auc:0.668128
[4]	eval-auc:0.670563	train-auc:0.671545
[5]	eval-auc:0.670288	train-auc:0.674189
[6]	eval-auc:0.669605	train-auc:0.676300
[7]	eval-auc:0.669348	train-auc:0.678100
[8]	eval-auc:0.669074	train-auc:0.679644
[9]	eval-auc:0.668972	train-auc:0.681049
[10]	eval-auc:0.668542	train-auc:0.682310
[11]	eval-auc:0.668477	train-auc:0.683454
[12]	eval-auc:0.668432	train-auc:0.684523
[13]	eval-auc:0.668209	train-auc:0.685519
[14]	eval-auc:0.667939	train-auc:0.686449
[15]	eval-auc:0.667826	train-auc:0.687337
[16]	eval-auc:0.667542	train-auc:0.688165
[17]	eval-auc:0.667360	train-auc:0.688977
[18]	eval-auc:0.667072	train-auc:0.689722
[19]	eval-auc:0.667002	train-auc:0.690430
[20]	eval-auc:0.666805	train-auc:0.691141
[21]	eval-auc:0.666720	train-auc:0.691807
[22]	eval-auc:0.666416	train-auc:0.692440
[23]	eval-auc:0.666250	train-auc:0.693043
[2

...


[0.6595429430941081, 0.65547968390468792, 0.64236845740901549, 0.66234286026161149, 0.65382586279251009, 0.66015725651403501, 0.67154353325498473, 0.66066954854561066]

[0]	eval-auc:0.636679	train-auc:0.649491
[1]	eval-auc:0.642446	train-auc:0.659745
[2]	eval-auc:0.645278	train-auc:0.666386
[3]	eval-auc:0.647381	train-auc:0.670784
[4]	eval-auc:0.648563	train-auc:0.673994
[5]	eval-auc:0.649170	train-auc:0.676688
[6]	eval-auc:0.649763	train-auc:0.678757
[7]	eval-auc:0.649964	train-auc:0.680571
[8]	eval-auc:0.650054	train-auc:0.682047
[9]	eval-auc:0.650025	train-auc:0.683389
[10]	eval-auc:0.649912	train-auc:0.684641
[11]	eval-auc:0.649944	train-auc:0.685715
[12]	eval-auc:0.650231	train-auc:0.686743
[13]	eval-auc:0.650464	train-auc:0.687698
[14]	eval-auc:0.650687	train-auc:0.688596
[15]	eval-auc:0.650858	train-auc:0.689458
[16]	eval-auc:0.650982	train-auc:0.690237
[17]	eval-auc:0.650975	train-auc:0.690971
[18]	eval-auc:0.651121	train-auc:0.691699
[19]	eval-auc:0.651263	train-auc:0.692397
[20]	eval-auc:0.651371	train-auc:0.693059
[21]	eval-auc:0.651460	train-auc:0.693682
[22]	eval-auc:0.651596	train-auc:0.694285
[23]	eval-auc:0.651679	train-auc:0.694847
[2


[0.6595429430941081, 0.65547968390468792, 0.64236845740901549, 0.66234286026161149, 0.65382586279251009, 0.66015725651403501, 0.67154353325498473, 0.66066954854561066, 0.65344273278633602]

[0]	eval-auc:0.616992	train-auc:0.650506
[1]	eval-auc:0.624515	train-auc:0.661289
[2]	eval-auc:0.628955	train-auc:0.667471
[3]	eval-auc:0.632393	train-auc:0.671821
[4]	eval-auc:0.634531	train-auc:0.675076
[5]	eval-auc:0.636565	train-auc:0.677674
[6]	eval-auc:0.638212	train-auc:0.679714
[7]	eval-auc:0.638787	train-auc:0.681441
[8]	eval-auc:0.639738	train-auc:0.682979
[9]	eval-auc:0.640186	train-auc:0.684341
[10]	eval-auc:0.640521	train-auc:0.685559
[11]	eval-auc:0.641164	train-auc:0.686678
[12]	eval-auc:0.641579	train-auc:0.687671
[13]	eval-auc:0.642112	train-auc:0.688639
[14]	eval-auc:0.642543	train-auc:0.689529
[15]	eval-auc:0.642711	train-auc:0.690396
[16]	eval-auc:0.643068	train-auc:0.691222
[17]	eval-auc:0.643291	train-auc:0.691992
[18]	eval-auc:0.643319	train-auc:0.692722
[19]	eval-auc:0.643685	train-auc:0.693402
[20]	eval-auc:0.643903	train-auc:0.694060
[21]	eval-auc:0.643946	train-auc:0.694694
[22]	eval-auc:0.644064	train-auc:0.695273
[23]	eval-auc:0.644326	train-auc:0.695854
[2


[0.6595429430941081, 0.65547968390468792, 0.64236845740901549, 0.66234286026161149, 0.65382586279251009, 0.66015725651403501, 0.67154353325498473, 0.66066954854561066, 0.65344273278633602, 0.64435299578661698]


In [17]:
#the average cross-validation AUC
np.asarray(scores).mean()

0.65637258743495164

In [19]:
#now all the models are inlcuded in list lms, now save it  
for i,lm in zip(range(len(lms)),lms):
    lm.dump_model('model.'+str(i))